Let's load the data from the csv just as in `dataset.ipynb`.

In [ ]:
import pandas as pd
import numpy as np

raw_data_file_name = "../dataset/fer2013.csv"
raw_data = pd.read_csv(raw_data_file_name)

Now, we separate and clean the data a little bit. First, we create an array of only the training data. Then, we create an array of only the private test data (referred to in the code with the prefix `first_test`). The `reset_index` call re-aligns the `first_test_data` to index from 0 instead of wherever it starts in the set.

In [ ]:
train_data = raw_data[raw_data["Usage"] == "Training"]

first_test_data = raw_data[raw_data["Usage"] == "PrivateTest"]
first_test_data.reset_index(inplace=True)

second_test_data = raw_data[raw_data["Usage"] == "PublicTest"]
second_test_data.reset_index(inplace=True)

In [ ]:
import keras

train_expected = keras.utils.to_categorical(train_data["emotion"], num_classes=7, dtype='int32')
first_test_expected = keras.utils.to_categorical(first_test_data["emotion"], num_classes=7, dtype='int32')
second_test_expected = keras.utils.to_categorical(second_test_data["emotion"], num_classes=7, dtype='int32')

In [ ]:
def process_pixels(array_input):
    output = np.empty([int(len(array_input)), 2304])
    for index, item in enumerate(output):
        item[:] = array_input[index].split(" ")
    output /= 255
    return output

In [ ]:
train_pixels = process_pixels(train_data["pixels"])
train_pixels = train_pixels.reshape(train_pixels.shape[0], 48, 48, 1)

In [ ]:
first_test_pixels = process_pixels(first_test_data["pixels"])
first_test_pixels = first_test_pixels.reshape(first_test_pixels.shape[0], 48, 48, 1)

In [ ]:
second_test_pixels = process_pixels(second_test_data["pixels"])
second_test_pixels = second_test_pixels.reshape(second_test_pixels.shape[0], 48, 48, 1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

Here, we create our own top-level network to load on top of VGG16.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, MaxPooling2D, Conv2D, Flatten
from keras.optimizers import Adam

def gen_model(size):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape = (48, 48, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())

    model.add(Dense(size, activation='relu'))
    model.add(Dense(7, activation='softmax'))

    optimizer = Adam(learning_rate=0.0009)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [ ]:
from keras.callbacks.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping('val_loss', patience=50)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(50/4), verbose=1)
callbacks = [early_stop, reduce_lr]

In [ ]:
sizes = [32, 64, 128, 256]
results = [None] * len(sizes)

for i in range(len(sizes)):
    model = gen_model(sizes[i])
    model.fit_generator(datagen.flow(train_pixels, train_expected, batch_size=32),
                    steps_per_epoch=len(train_pixels) / 32,
                    epochs=10, verbose=1, callbacks=callbacks,
                    validation_data=(first_test_pixels,first_test_expected))
    results[i] = model.evaluate(second_test_pixels, second_test_pixels, batch_size=32)